In [74]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # the for of in with
from nltk.stem.porter import PorterStemmer # loved loving == love
from sklearn.feature_extraction.text import TfidfVectorizer # loved = [0.0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [75]:
news_df = pd.read_csv('train.csv')

In [76]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [77]:
news_df.shape

(20800, 5)

In [78]:
news_df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [79]:
news_df = news_df.fillna(' ')

In [80]:
news_df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [81]:
news_df['content'] = news_df['author']+" "+news_df['title']

In [82]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [105]:
news_df['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [106]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]'," ",content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [107]:
news_df['content'] = news_df['content'].apply(stemming)

In [108]:
news_df['content']

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [109]:
X = news_df['content'].values
y = news_df['label'].values

In [110]:
print(X)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [111]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [112]:
print(X)

  (0, 15553)	0.28485063562728646
  (0, 13348)	0.2565896679337957
  (0, 8822)	0.3635963806326075
  (0, 8546)	0.29212514087043684
  (0, 7612)	0.24785219520671603
  (0, 6930)	0.21874169089359144
  (0, 4917)	0.233316966909351
  (0, 3759)	0.2705332480845492
  (0, 3567)	0.3598939188262559
  (0, 2933)	0.2468450128533713
  (0, 2464)	0.3676519686797209
  (0, 263)	0.27010124977708766
  (1, 16656)	0.3025156488372128
  (1, 6744)	0.19152496072048605
  (1, 5440)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (1, 2790)	0.19208753385709676
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (1, 1481)	0.2957471154505952
  (2, 15478)	0.41544962664721613
  (2, 9532)	0.49351492943649944
  (2, 5903)	0.3474613386728292
  (2, 5326)	0.3866530551182615
  (2, 3072)	0.46097489583229645
  :	:
  (20797, 12998)	0.2483705036831893
  (20797, 12227)	0.27276402145717243
  (20797, 12022)	0.24790022252744132
  (20797, 10216)	0.0804189541935242
  (20797, 9500)	0.17463635692029988
  (20797, 9431)	0.29394

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [114]:
X_train.shape

(16640, 16984)

In [115]:
X_test.shape

(4160, 16984)

In [116]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [117]:
train_y_pred = model.predict(X_train)
print("train accurracy :",accuracy_score(train_y_pred,y_train))

train accurracy : 0.9868990384615385


In [118]:
test_y_pred = model.predict(X_test)
print("train accurracy :",accuracy_score(test_y_pred,y_test))

train accurracy : 0.9764423076923077


In [119]:
# prediction system

input_data = X_test[20]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake news')
else:
    print('Real news')

Fake news


In [120]:
news_df['content'][20]

'news hope gop nude paul ryan emerg ayahuasca tent vision new republican parti'

Note: you may need to restart the kernel to use updated packages.
